In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/content/formspring_data.csv',sep='\t')

In [3]:
selector1 = df['ans1'] == df['ans2']
selector2 = df['ans2'] == df['ans3']
selector3 = df['ans3'] == df['ans1']

In [4]:
df = df[selector1 & selector2 & selector3]

In [5]:
df = df[['ques','ans1']]

In [6]:
df['ans1'] = df['ans1'].replace(("Yes","No"),(1,0))

In [7]:
df = df.astype(str)

In [8]:
corpus = []
from nltk.corpus import stopwords
import re
def clean_data(sentence):
    sentence = sentence.replace('&#039;','"').replace("<br>"," ").replace("&quot;","'")
    sentence = re.sub('[^a-zA-Z]',' ',sentence)
    sentence = sentence.lower().split()
    words = [word for word in sentence if not word in stopwords.words('english')]
    sentence = ' '.join(words)
    corpus.append(sentence)

In [9]:
df.shape

(11089, 2)

In [10]:
df.columns = ['Comment','Cyberbullying?']

In [13]:
import nltk
nltk.download('stopwords')
for sentence in df['Comment']:
    clean_data(sentence)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
len(corpus)

11089

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()


In [16]:
from sklearn.model_selection import train_test_split
y = df.iloc[:,-1].values.astype(int)
print(y[0:5])
X_train,X_test,y_train,y_test = train_test_split(corpus,y,test_size=0.2,random_state=0)

[0 0 0 0 0]


In [17]:
cv.fit(X_train)
X_train = cv.transform(X_train)
X_test = cv.transform(X_test)

In [18]:
X_train = X_train.toarray()

In [19]:
X_test = X_test.toarray()

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [21]:
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [22]:
from sklearn.metrics import accuracy_score,f1_score,recall_score

In [23]:
y_pred = clf.predict(X_test)

In [24]:
score = accuracy_score(y_test,y_pred)

In [26]:
f1_score__ = f1_score(y_test,y_pred)
recall = recall_score(y_test,y_pred)

In [ ]:
# function to use for prediction
# firstly have to fit pickled model on vocabulary(X_train precisely) ==> clf.fit(X_train)
def pre(words):
    words = cv.transform(words).toarray()
    prediction = clf.predict(words)
    return prediction

In [27]:
print("Accuracy => ",score)
print("Recall => ",recall)
print("F1 score => ",f1_score__)

Accuracy =>  0.9670874661857529
Recall =>  0.38461538461538464
F1 score =>  0.4065040650406504


In [29]:
import joblib
joblib.dump(clf,'basic_model.pkl')

['basic_model.pkl']